In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import argrelextrema
from sklearn.preprocessing import StandardScaler


In [2]:
df = pd.read_csv('../../data/H161.csv', parse_dates = ["utc_time"])
df["utc_time"] = df["utc_time"].dt.tz_localize(None)

df.columns = ['time','phase', 'step', 'statement', 'heat_time', 'weight', 'temp', 'pressure']

FileNotFoundError: [Errno 2] No such file or directory: '../../data/H161.csv'

In [ ]:
df = df.drop(['statement','step'], axis=1)

In [ ]:
df.sample(40)

In [ ]:
df = df[df['phase'].str.contains ('DRYING|DISCHRG') == True]

In [ ]:
df.replace({'????????': None}, inplace=True)
df.dropna(inplace=True)

In [ ]:
df['temp'] = df.temp.astype(float)
df['weight'] = df.weight.astype(float)
df['pressure'] = df.pressure.astype(float)
df['heat_time'] = df.heat_time.astype(float)
df['phase'] = df.phase.astype('category')
# df.set_index('time', inplace=True)


In [ ]:
n = 180  # number of points to be checked before and after

# Find local peaks
df['max_temp'] = 0
df['max_temp'] = df.iloc[argrelextrema(df.temp.values, np.greater_equal, order=n)]['temp']

In [ ]:
df['ready'] = 0

In [ ]:
for i in range (1, len(df['ready']-1)):
    if df.max_temp.values[i] > 0:
            while df['phase'].values[i] == 'DRYING':
                i += 1
                df.ready.values[i] = 1
df = df.drop(['max_temp'], axis=1)

In [ ]:
df.head(300)

In [ ]:
# Labeel encoding
df['phase'] = df['phase'].cat.codes
df

In [ ]:
df['p_lagged_30'] = df['pressure'].shift(-30)
df['t_lagged_30'] = df['temp'].shift(-30)
df['w_lagged_30'] = df['weight'].shift(-30)
df['ps_lagged_30'] = df['phase'].shift(-30)
df['ht_lagged_30'] = df['heat_time'].shift(-30)

df['p_lagged_25'] = df['pressure'].shift(-25)
df['t_lagged_25'] = df['temp'].shift(-25)
df['w_lagged_25'] = df['weight'].shift(-25)
df['ps_lagged_25'] = df['phase'].shift(-25)
df['ht_lagged_25'] = df['heat_time'].shift(-25)

In [ ]:
X = df.values[:,:-1]
y = df.iloc[:, -1].values

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)
print(X_train)
print(y_train)
print(X_test)
print(y_test)


In [ ]:
df.tail(40)